In [ ]:
from manim import *
import textwrap
import numpy as np

class Manim3DDeepdive(ThreeDScene):
    def construct(self):
        # ==========================================
        # 0. CONFIGURATION
        # ==========================================
        # Shift Frame LEFT so Origin (0,0,0) appears on the RIGHT
        self.frame_shift = LEFT * 3.5
        self.camera_zoom = 0.65 
        
        # UI Background Position
        self.ui_bg_pos = LEFT * 6.5 

        # Initialize UI (The "Code Editor" on the left)
        self.setup_fixed_ui()
        
        # Initialize Camera (Standard Front/Side View)
        self.set_camera_orientation(
            phi=75 * DEGREES,      # Tilt (0=Top down, 90=Front)
            theta=-45 * DEGREES,   # Spin (Angle around Z)
            zoom=self.camera_zoom,
            frame_center=self.frame_shift
        )

        # ==========================================
        # 1. AXES SETUP
        # ==========================================
        self.run_axes_setup()

        # ==========================================
        # 2. CAMERA MOVEMENT DEEP DIVE
        # ==========================================
        # We will show Tilt, Spin, and Zoom separately
        self.run_camera_deep_dive()

        # ==========================================
        # 3. POSITIONING DEEP DIVE
        # ==========================================
        # We will show X, Y, and Z movement individually
        self.run_positioning_deep_dive()

        # ==========================================
        # 4. CUBE ROTATION
        # ==========================================
        self.run_cube_demo()

        # ==========================================
        # 5. PARAMETRIC SURFACE
        # ==========================================
        self.run_surface_demo()

        # ==========================================
        # 6. PARAMETRIC PATH
        # ==========================================
        self.run_path_demo()

        # End
        self.update_code("# Tutorial Complete.")
        self.wait(3)

    # ---------------------------------------------------------
    # UI & HELPER METHODS
    # ---------------------------------------------------------
    def setup_fixed_ui(self):
        # 1. Black Background Mask
        self.ui_bg = Rectangle(
            width=7.0, height=10, 
            color=BLACK, fill_color=BLACK, fill_opacity=1, stroke_width=0
        )
        self.ui_bg.move_to(self.ui_bg_pos)
        self.ui_bg.set_z_index(10)
        
        # 2. Editor Border
        self.code_border = Rectangle(
            width=6.0, height=7.0, color=GRAY, stroke_width=2
        )
        self.code_border.move_to(self.ui_bg_pos)
        self.code_border.set_z_index(11)
        
        # 3. Header Text
        self.title = Text("3D Masterclass", font_size=36)
        self.title.to_edge(UP, buff=0.2)
        self.title.set_z_index(12)
        
        self.t_code = Text("logic.py", font_size=20, color=BLUE)
        self.t_code.next_to(self.code_border, UP, aligned_edge=LEFT)
        self.t_code.set_z_index(12)
        
        # Attach to Camera Frame
        self.add_fixed_in_frame_mobjects(
            self.ui_bg, self.code_border, self.title, self.t_code
        )

    def update_code(self, code_text):
        if hasattr(self, 'code_group'):
            self.remove_fixed_in_frame_mobjects(self.code_group)
            self.remove(self.code_group)
            
        lines = textwrap.dedent(code_text).strip().split('\n')
        self.code_group = VGroup()
        
        start_point = self.code_border.get_top() + DOWN * 0.5 + LEFT * 2.6
        
        for i, line in enumerate(lines):
            # Syntax Highlighting
            c = WHITE
            if line.strip().startswith("#"): c = GREY
            elif "class" in line or "def" in line: c = ORANGE
            elif "move_camera" in line or "play" in line: c = BLUE_B
            elif "phi" in line or "theta" in line: c = YELLOW
            elif "UP" in line or "RIGHT" in line or "OUT" in line: c = RED
            
            t = Text(line, font="Monospace", font_size=16, color=c)
            t.move_to(start_point + DOWN * (i * 0.35), aligned_edge=LEFT)
            self.code_group.add(t)
            
        self.code_group.set_z_index(13)
        self.add_fixed_in_frame_mobjects(self.code_group)

    # ---------------------------------------------------------
    # CHAPTER 1: AXES
    # ---------------------------------------------------------
    def run_axes_setup(self):
        self.update_code("""
        # 1. Setup Axes
        axes = ThreeDAxes()
        
        # Labels: X, Y, Z
        lab = axes.get_axis_labels()
        
        self.add(axes, lab)
        """)
        
        self.axes = ThreeDAxes()
        self.axes_origin = ORIGIN - LEFT * 3.5 
        self.axes.move_to(self.axes_origin)
        
        x_lbl = self.axes.get_x_axis_label("X").set_color(RED)
        y_lbl = self.axes.get_y_axis_label("Y").set_color(GREEN)
        z_lbl = self.axes.get_z_axis_label("Z").set_color(BLUE)
        self.axis_labels = VGroup(x_lbl, y_lbl, z_lbl)

        self.play(Create(self.axes), FadeIn(self.axis_labels))
        self.wait(1)

    # ---------------------------------------------------------
    # CHAPTER 2: CAMERA MOVEMENT (ALL PATHS)
    # ---------------------------------------------------------
    def run_camera_deep_dive(self):
        # Setup a reference object
        self.sphere = Sphere(radius=1.5).set_fill(BLUE, 0.5).set_stroke(BLUE_E, 1)
        self.play(GrowFromCenter(self.sphere))
        
        # --- 2A. THETA (Spinning) ---
        self.update_code("""
        # 2A. Camera Spin (Theta)
        # Rotates around Z-axis.
        # "Orbiting" the object.
        
        self.move_camera(
            theta = -135 * DEGREES
        )
        """)
        self.move_camera(theta=-135*DEGREES, run_time=3)
        self.wait(0.5)
        
        # --- 2B. PHI (Tilting) ---
        self.update_code("""
        # 2B. Camera Tilt (Phi)
        # 0 = Top Down
        # 90 = Side View
        
        self.move_camera(
            phi = 30 * DEGREES
        )
        """)
        self.move_camera(phi=30*DEGREES, run_time=3)
        self.wait(0.5)
        
        # --- 2C. ZOOM (Dolly) ---
        self.update_code("""
        # 2C. Zoom
        # Moving camera closer.
        
        self.move_camera(
            zoom = 1.0
        )
        """)
        self.move_camera(zoom=1.0, run_time=2)
        self.wait(0.5)
        
        # --- RESET ---
        self.update_code("# Resetting View...")
        self.move_camera(
            phi=75*DEGREES, theta=-45*DEGREES, zoom=self.camera_zoom,
            frame_center=self.frame_shift, run_time=1.5
        )
        self.play(FadeOut(self.sphere))

    # ---------------------------------------------------------
    # CHAPTER 3: POSITIONING (EVERY DIRECTION)
    # ---------------------------------------------------------
    def run_positioning_deep_dive(self):
        dot = Sphere(radius=0.2, color=YELLOW)
        
        # --- 3A. X-AXIS ---
        self.update_code("""
        # 3A. X-Axis Movement
        # RIGHT vector = [1, 0, 0]
        
        dot.move_to(RIGHT * 2)
        """)
        # We use simple lines to visualize the 'force' moving the dot
        arrow_x = Arrow(self.axes_origin, RIGHT*2, color=RED, buff=0)
        self.play(FadeIn(dot))
        self.play(Create(arrow_x))
        self.play(dot.animate.move_to(RIGHT*2))
        self.play(FadeOut(arrow_x))
        
        # --- 3B. Y-AXIS ---
        self.update_code("""
        # 3B. Y-Axis Movement
        # UP vector = [0, 1, 0]
        
        dot.shift(UP * 2)
        """)
        # Start arrow from current dot position
        start = dot.get_center()
        arrow_y = Arrow(start, start + UP*2, color=GREEN, buff=0)
        self.play(Create(arrow_y))
        self.play(dot.animate.shift(UP*2))
        self.play(FadeOut(arrow_y))
        
        # --- 3C. Z-AXIS ---
        self.update_code("""
        # 3C. Z-Axis Movement
        # OUT vector = [0, 0, 1]
        
        dot.shift(OUT * 2)
        """)
        start = dot.get_center()
        arrow_z = Arrow(start, start + OUT*2, color=BLUE, buff=0)
        # Rotate arrow to point OUT (Z-axis)
        arrow_z.rotate(90*DEGREES, axis=RIGHT) 
        
        self.play(Create(arrow_z))
        self.play(dot.animate.shift(OUT*2))
        self.play(FadeOut(arrow_z))
        
        self.update_code("# Final Pos: (2, 2, 2)")
        self.wait(1)
        self.play(FadeOut(dot))

    # ---------------------------------------------------------
    # CHAPTER 4: CUBE ROTATION
    # ---------------------------------------------------------
    def run_cube_demo(self):
        self.update_code("""
        # 4. Object Rotation
        # We rotate the CUBE,
        # not the camera.
        
        cube.rotate(
            angle=PI/2, 
            axis=UP
        )
        """)
        
        cube = Cube(side_length=2, fill_opacity=0.5, stroke_width=2, stroke_color=WHITE)
        cube.set_fill(color=PURPLE)
        self.play(FadeIn(cube))
        
        # Explicit rotation animation
        self.play(Rotate(cube, angle=PI/2, axis=UP), run_time=2)
        self.play(Rotate(cube, angle=PI/2, axis=RIGHT), run_time=2)
        
        self.play(FadeOut(cube))

    # ---------------------------------------------------------
    # CHAPTER 5: PARAMETRIC SURFACE
    # ---------------------------------------------------------
    def run_surface_demo(self):
        self.update_code("""
        # 5. Parametric Surface
        # u, v -> x, y, z
        
        Surface(
           lambda u, v: [
              u, v, sin(u)*cos(v)
           ]
        )
        """)
        
        # Define a surface (Ripple)
        surface = Surface(
            lambda u, v: self.axes.c2p(
                u, v, 0.5 * np.sin(2*u) * np.cos(2*v)
            ),
            u_range=[-2, 2],
            v_range=[-2, 2],
            resolution=(24, 24),
            should_make_jagged=False
        )
        
        surface.set_style(fill_opacity=0.8, stroke_color=WHITE, stroke_width=0.2)
        surface.set_fill_by_checkerboard(TEAL, TEAL_E)
        
        self.play(Create(surface), run_time=3)
        
        # Slight rotation to see depth
        self.play(Rotate(surface, angle=PI/4, axis=UP))
        self.play(FadeOut(surface))

    # ---------------------------------------------------------
    # CHAPTER 6: PARAMETRIC PATH
    # ---------------------------------------------------------
    def run_path_demo(self):
        self.update_code("""
        # 6. Parametric Path
        # Spiral: cos(t), sin(t), t
        
        path = ParametricFunction(...)
        
        self.play(
           MoveAlongPath(dot, path)
        )
        """)
        
        # 1. Create Curve
        curve = ParametricFunction(
            lambda t: self.axes.c2p(
                1.5 * np.cos(t), 
                1.5 * np.sin(t), 
                t / 2
            ),
            t_range=[-2*PI, 2*PI],
            color=ORANGE
        )
        
        # 2. Create Object
        dot = Sphere(radius=0.2, color=YELLOW)
        dot.move_to(curve.get_start())
        
        self.play(Create(curve), FadeIn(dot))
        self.play(MoveAlongPath(dot, curve), run_time=4, rate_func=linear)
        self.play(FadeOut(curve), FadeOut(dot))

%manim -ql -v warning Manim3DDeepdive